This document parses and transforms CSV High Frontier 4 patent cards into JSON.

# Preamble

## CSV import

In [34]:
import csv
import glob
from os.path import splitext, basename
import unittest

def read_cards(deckname, fields):
    with open(f'decks/{deckname}.csv', 'r') as fh:
        reader = csv.DictReader(fh, fieldnames=fields)
        next(reader) # Drop row 1, which contains multi-collumn cells describing groups of fields (e.g. Thruster and Support Requirements)
        next(reader) # Drop row 2, which contains field names
        deck = list(reader)
        
    for i, card in enumerate(deck):
        split_keys(card)
        card['Deck'] = deckname
        card['Side'] = side_from_index(i)
                
    return deck    

def side_from_index(i):
    if i % 2 == 0:
        return 'Front'
    return 'Reverse'

def split_keys(card):
    namespaced_keys = [k for k in card if '.' in k]
    for key in namespaced_keys:
        k0, k1 = key.split('.')
        if not card.get(k0):
            card[k0] = {}
        card[k0][k1] = card.pop(key)

def chunklist(l, n):
    return zip(*[iter(l)] * 2)
               
def build_deck(deckname, clean_cards):
    return {
        'Name': deckname,
        'Patents': [
            {
                'Deck': deckname,
                'Front': front,
                'Reverse': reverse,
            }
            for front, reverse in chunklist(clean_cards, 2)
        ]
    }

class TestUtils(unittest.TestCase):
    def test_split(self):
        card = {
            'Mass': 0,
            'Rad Hard': 4,
            'Thrust Triangle.Thrust': 3,
            'Thrust Triangle.Fuel Consumption': '2',
            'Support Requirements.⟛ Generator': '1'
        }
        
        expected_result = {
            'Mass': 0,
            'Rad Hard': 4,
            'Thrust Triangle': {
                'Thrust': 3,
                'Fuel Consumption': '2',
            },
            'Support Requirements': {'⟛ Generator': '1'},
        }
        
        split_keys(card)
        self.assertEqual(card, expected_result)
        
    def test_build_deck(self):
        cards = [1, 2, 3, 4, 5, 6]
        
        deck = build_deck('Test', cards)
        expected_deck = {
            'Name': 'Test',
            'Patents': [
                { 'Deck': 'Test', 'Front': 1, 'Reverse': 2 },
                { 'Deck': 'Test', 'Front': 3, 'Reverse': 4 },
                { 'Deck': 'Test', 'Front': 5, 'Reverse': 6 },
            ]
        }
    
        self.assertEqual(deck, expected_deck)
        
unittest.main(argv=['', 'TestUtils'], verbosity=2, exit=False)

test_build_deck (__main__.TestUtils) ... ok
test_split (__main__.TestUtils) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.001s

OK


## Support requirement parsing

In [2]:
import unittest

def supports(card):
    supports = []
    
    # Special case for Colliding FRC 3He-D Fusion TeraWatt Thruster
    if card.get('Support Requirements', {}).get('Any Reactor', False):
        any_reactor = {'Type': 'Reactor', 'Subtypes': ['∿', '💣', 'X']}
        supports += [any_reactor, any_reactor]      

    reactor_subtypes = []    
    # Other than the Colliding FRC 3He-D Fusion TeraWatt Thruster, cards only need 1 reactor symbol. If we have multiple symbols, that means that we need any one of them, not several.
    for t in ['∿', '💣', 'X']:
        if card.get('Support Requirements', {}).get(f'{t} Reactor', False):
            reactor_subtypes.append(t)

    if len(reactor_subtypes) != 0:
        supports.append(
            {
                'Type': 'Reactor',
                'Subtypes': reactor_subtypes
            }
        )

    # Cards only ever need exactly one generator type
    if card.get('Support Requirements', {}).get('e Generator', False):
        supports.append(
            {
                'Type': 'Generator',
                'Subtypes': ['e']
            }
        )
    elif card.get('Support Requirements', {}).get('⟛ Generator', False):
        supports.append(
            {
                'Type': 'Generator',
                'Subtypes': ['⟛']
            }
        )

    therms = card.get('Support Requirements', {}).get('Cooling', 0)
    if therms != 0:
        supports.append({'Type': 'Cooling', 'Therms': therms})
    
    return supports

class SupportTests(unittest.TestCase):
    
    def test_no_supports(self):
        card = {}
        self.assertEqual(supports(card), [])
        
    def test_single_generator(self):
        card = {'Support Requirements': {'⟛ Generator': True}}
        self.assertEqual(supports(card), [{'Type': 'Generator', 'Subtypes': ['⟛']}])

    def test_single_reactor(self):
        card = {'Support Requirements': {'💣 Reactor': True}}
        self.assertEqual(supports(card), [{'Type': 'Reactor', 'Subtypes': ['💣']}])
        
    def test_multiple_reactors(self):
        card = {'Support Requirements': {'💣 Reactor': True, '∿ Reactor': True, 'X Reactor': True}}
        self.assertEqual(supports(card), [{
            'Type': 'Reactor',
            'Subtypes': ['∿', '💣', 'X']
        }])

    def test_reactor_and_generator(self):
        card = {'Support Requirements': {'💣 Reactor': True, 'X Reactor': False, 'e Generator': True}}
        self.assertEqual(supports(card), [{'Type': 'Reactor', 'Subtypes': ['💣']}, {'Type': 'Generator', 'Subtypes': ['e']}])
        
    def test_therms(self):
        card = {'Support Requirements': {'Cooling': 2}}
        self.assertEqual(supports(card), [{'Type': 'Cooling', 'Therms': 2}])
    
    def test_colliding_frc_special_case(self):
        '''
        The Colliding FRC 3He-D Fusion GW Thruster has the 'Any' reactor subtype in the spreadsheet. It actually requires TWO of any reactor. We handle this as a special case which requires two Reactor type support requirements.
        '''
        card = {'Support Requirements': {'Any Reactor': True}}
        any_reactor = {'Type': 'Reactor', 'Subtypes': ['∿', '💣', 'X']}
        self.assertEqual(supports(card), [any_reactor, any_reactor])
        

unittest.main(argv=['', 'SupportTests'], verbosity=2, exit=False)

test_colliding_frc_special_case (__main__.SupportTests)
The Colliding FRC 3He-D Fusion GW Thruster has the 'Any' reactor subtype in the spreadsheet. It actually requires TWO of any reactor. We handle this as a special case which requires two Reactor type support requirements. ... ok
test_multiple_reactors (__main__.SupportTests) ... ok
test_no_supports (__main__.SupportTests) ... ok
test_reactor_and_generator (__main__.SupportTests) ... ok
test_single_generator (__main__.SupportTests) ... ok
test_single_reactor (__main__.SupportTests) ... ok
test_therms (__main__.SupportTests) ... ok

----------------------------------------------------------------------
Ran 7 tests in 0.004s

OK


## Debug tools

In [3]:
import random

def draw(deck):
    return random.choice(deck)

# Card imports

## Thrusters

In [41]:
'''
Thrusters
'''

fields = [
    'Name',
    'Spectral Type',
    'Mass',
    'Rad-Hard',
    'Thrust Triangle.Thrust',
    'Thrust Triangle.Fuel Consumption',
    'Thrust Triangle.Fuel Type',
    'Bonus Pivots',
    'Thrust Triangle.Afterburn',
    'Thrust Triangle.Push',
    'Thrust Triangle.Solar',
    'Support Requirements.⟛ Generator',
    'Support Requirements.e Generator',
    'Support Requirements.X Reactor',
    'Support Requirements.∿ Reactor',
    'Support Requirements.💣 Reactor',
    'Support Requirements.Cooling',
    'Ability',
]

int_fields = [
    'Mass',
    'Rad-Hard',
    'Thrust Triangle.Thrust',
    'Thrust Triangle.Afterburn',
    'Support Requirements.Cooling',
    'Bonus Pivots',
]

bool_fields = [
    'Thrust Triangle.Push',
    'Thrust Triangle.Solar',
    'Support Requirements.⟛ Generator',
    'Support Requirements.e Generator',
    'Support Requirements.X Reactor',
    'Support Requirements.∿ Reactor',
    'Support Requirements.💣 Reactor',
]

def str_to_int(s):
    if s == '':
        return 0
    else:
        return int(s)

def str_to_bool(s):
    if s == '0' or s == '':
        return False
    else:
        return True

def intify(card, fields):
    for k in fields:
        if '.' in k:
            k0, k1 = k.split('.')
            card[k0][k1] = str_to_int(card[k0][k1])
        else:
            card[k] = str_to_int(card[k])

def boolify(card, fields):
    for k in fields:
        if '.' in k:
            k0, k1 = k.split('.')
            card[k0][k1] = str_to_bool(card[k0][k1])
        else:
            card[k] = str_to_bool(card[k])

thrusters = read_cards('Thrusters', fields)
for i, card in enumerate(thrusters):
    if i % 2 == 0:
        spectral_type = card.get('Spectral Type')
    card['Spectral Type'] = spectral_type
    intify(card, int_fields)
    boolify(card, bool_fields)
    card['Support Requirements'] = supports(card)
    
thruster_deck = build_deck('Thrusters', thrusters)

## Robonauts

In [5]:
import unittest

def isru(robonaut):
    return {
        'Rating': robonaut['ISRU']['ISRU'],
        'Platforms': [p for p in ['Raygun', 'Buggy', 'Missile'] if robonaut.get('ISRU').get(p)],
    }

class ISRUTests(unittest.TestCase):
    
    def test_raygun(self):
        card = {'ISRU': {'ISRU': 2, 'Raygun': True, 'Buggy': False, 'Missile': False}}
        self.assertEqual(isru(card), {'Rating': 2, 'Platforms': ['Raygun']})
    
    def test_buggy(self):
        card = {'ISRU': {'ISRU': 0, 'Raygun': False, 'Buggy': True, 'Missile': False}}
        self.assertEqual(isru(card), {'Rating': 0, 'Platforms': ['Buggy']})
        
    def test_missile(self):
        card = {'ISRU': {'ISRU': 1, 'Raygun': False, 'Buggy': False, 'Missile': True}}
        self.assertEqual(isru(card), {'Rating': 1, 'Platforms': ['Missile']})

    def test_multiple(self):
        card = {'ISRU': {'ISRU': 1, 'Raygun': True, 'Buggy': False, 'Missile': True}}
        self.assertEqual(isru(card), {'Rating': 1, 'Platforms': ['Raygun', 'Missile']})

unittest.main(argv=['', 'ISRUTests'], verbosity=2, exit=False)

test_buggy (__main__.ISRUTests) ... ok
test_missile (__main__.ISRUTests) ... ok
test_multiple (__main__.ISRUTests) ... ok
test_raygun (__main__.ISRUTests) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.003s

OK


In [6]:
'''
Robonauts
'''

fields = [
    'Name',
    'Spectral Type',
    'Mass',
    'Rad-Hard',
    'Thrust Triangle.Thrust',
    'Thrust Triangle.Fuel Consumption',
    'Thrust Triangle.Fuel Type',
    'Thrust Triangle.Afterburn',
    'Thrust Triangle.Push',
    'Thrust Triangle.Solar',
    'ISRU.ISRU',
    'ISRU.Missile',
    'ISRU.Raygun',
    'ISRU.Buggy',
    'Support Requirements.⟛ Generator',
    'Support Requirements.e Generator',
    'Support Requirements.X Reactor',
    'Support Requirements.∿ Reactor',
    'Support Requirements.💣 Reactor',
    'Support Requirements.Cooling',
    'Ability',
]

int_fields = [
    'Mass',
    'Rad-Hard',
    'Thrust Triangle.Thrust',
    'Thrust Triangle.Afterburn',
    'Support Requirements.Cooling',
    'ISRU.ISRU',
]

bool_fields = [
    'Thrust Triangle.Push',
    'Thrust Triangle.Solar',
    'Support Requirements.⟛ Generator',
    'Support Requirements.e Generator',
    'Support Requirements.X Reactor',
    'Support Requirements.∿ Reactor',
    'Support Requirements.💣 Reactor',
    'ISRU.Missile',
    'ISRU.Buggy',
    'ISRU.Raygun',
 ]

def has_thruster(card):
    return card.get('Thrust Triangle', {}).get('Thrust', '') != ''

robonauts = read_cards('Robonauts', fields)
for card in robonauts:
    if has_thruster(card):
        intify(card, int_fields)
        boolify(card, bool_fields)
    else:
        card.pop('Thrust Triangle')
        intify(card, filter(lambda f: not f.startswith('Thrust Triangle.'), int_fields))
        boolify(card, filter(lambda f: not f.startswith('Thrust Triangle.'), bool_fields))
    card['Support Requirements'] = supports(card)
    card['ISRU'] = isru(card)
    
draw(robonauts)

{'Name': 'Electrophoretic Sandworm',
 'Spectral Type': '',
 'Mass': 1,
 'Rad-Hard': 5,
 'Ability': '',
 'ISRU': {'Rating': 1, 'Platforms': ['Buggy']},
 'Support Requirements': [],
 'Deck': 'Robonauts',
 'Side': 'Reverse'}

## Refineries

In [7]:
'''
Refineries
'''

fields = [
    'Name',
    'Spectral Type',
    'Mass',
    'Rad-Hard',
    'Air Eater',
    'Support Requirements.e Generator',
    'Support Requirements.X Reactor',
    'Support Requirements.∿ Reactor',
    'Support Requirements.💣 Reactor',
    'Ability',
]

int_fields = [
    'Mass',
    'Rad-Hard',
]

bool_fields = [
    'Air Eater',
    'Support Requirements.e Generator',
    'Support Requirements.X Reactor',
    'Support Requirements.∿ Reactor',
    'Support Requirements.💣 Reactor',
]

refineries = read_cards('Refineries', fields)
for card in refineries:
    intify(card, int_fields)
    boolify(card, bool_fields)
    card['Support Requirements'] = supports(card)
    
draw(refineries)

{'Name': 'Ilmenite Semiconductor Film',
 'Spectral Type': '',
 'Mass': 2,
 'Rad-Hard': 6,
 'Air Eater': False,
 'Ability': 'POWER GIRDLE: If used to industrialize a non-atmospheric site of size 8+, you permanently gain the Powersat faction privilege.',
 'Support Requirements': [],
 'Deck': 'Refineries',
 'Side': 'Reverse'}

## Reactors

In [8]:
'''
Reactors
'''

fields = [
    'Name',
    'Spectral Type',
    'Mass',
    'Rad-Hard',
    'Supports.X',
    'Supports.∿',
    'Supports.💣',
    'Movement Modifiers.Thrust',
    'Movement Modifiers.Fuel Consumption',
    'Air Eater',
    'Support Requirements.⟛ Generator',
    'Support Requirements.X Reactor',
    'Support Requirements.Cooling',
    'Ability',
]

int_fields = [
    'Mass',
    'Rad-Hard',
    'Movement Modifiers.Thrust',
    'Support Requirements.Cooling',
]

bool_fields = [
    'Supports.X',
    'Supports.∿',
    'Supports.💣',
    'Air Eater',
    'Support Requirements.⟛ Generator',
    'Support Requirements.X Reactor',
]

def has_movement_modifiers(card):
    return card.get('Movement Modifiers', {}).get('Thrust', '') != ''

reactors = read_cards('Reactors', fields)
for card in reactors:
    if has_movement_modifiers(card):
        intify(card, int_fields)
        boolify(card, bool_fields)
    else:
        card.pop('Movement Modifiers')
        intify(card, filter(lambda f: not f.startswith('Movement Modifiers.'), int_fields))
        boolify(card, filter(lambda f: not f.startswith('Movement Modifiers.'), bool_fields))
    card['Support Requirements'] = supports(card)
    card['Supports'] = [k for k in card['Supports'].keys() if card['Supports'][k]]

draw(reactors)

{'Name': 'Pebble Bed Fission',
 'Spectral Type': 'S',
 'Mass': 1,
 'Rad-Hard': 6,
 'Air Eater': False,
 'Ability': '',
 'Supports': ['∿'],
 'Movement Modifiers': {'Thrust': 4, 'Fuel Consumption': '1/1'},
 'Support Requirements': [{'Type': 'Cooling', 'Therms': 1}],
 'Deck': 'Reactors',
 'Side': 'Front'}

## Radiators

In [9]:
'''
Radiators
'''

fields = [
    'Name',
    'Spectral Type',
    'Light Side.Mass',
    'Light Side.Rad-Hard',
    'Light Side.Therms',
    'Heavy Side.Mass',
    'Heavy Side.Rad-Hard',
    'Heavy Side.Therms',
    'Support Requirements.e Generator',
    'Ability',
]

int_fields = [
    'Light Side.Mass',
    'Light Side.Rad-Hard',
    'Light Side.Therms',
    'Heavy Side.Mass',
    'Heavy Side.Rad-Hard',
    'Heavy Side.Therms',
]

bool_fields = [
    'Support Requirements.e Generator',
]

radiators = read_cards('Radiators', fields)
for card in radiators:
    intify(card, int_fields)
    boolify(card, bool_fields)
    # The spreadsheet has a single support requirements field but technically the light and heavy side requirements are distinct, so we duplicate it
    support_reqs = supports(card)
    card.pop('Support Requirements')
    card['Light Side']['Support Requirements'] = support_reqs
    card['Heavy Side']['Support Requirements'] = support_reqs

draw(radiators)

{'Name': 'Ti / K Heat Pipe',
 'Spectral Type': 'M',
 'Ability': '',
 'Light Side': {'Mass': 2,
  'Rad-Hard': 3,
  'Therms': 1,
  'Support Requirements': []},
 'Heavy Side': {'Mass': 4,
  'Rad-Hard': 2,
  'Therms': 2,
  'Support Requirements': []},
 'Deck': 'Radiators',
 'Side': 'Front'}

## Generators

In [10]:
'''
Generators
'''

fields = [
    'Name',
    'Spectral Type',
    'Mass',
    'Rad-Hard',
    'Supports.⟛',
    'Supports.e',
    'Movement Modifiers.Thrust',
    'Movement Modifiers.Fuel Consumption',
    'Air Eater',
    'Movement Modifiers.Solar',
    'Support Requirements.e Generator',
    'Support Requirements.X Reactor',
    'Support Requirements.∿ Reactor',
    'Support Requirements.💣 Reactor',
    'Support Requirements.Cooling',
    'Ability',
]

int_fields = [
    'Mass',
    'Rad-Hard',
    'Movement Modifiers.Thrust',
    'Support Requirements.Cooling',
]

bool_fields = [
    'Supports.⟛',
    'Supports.e',
    'Air Eater',
    'Movement Modifiers.Solar',
    'Support Requirements.e Generator',
    'Support Requirements.X Reactor',
    'Support Requirements.∿ Reactor',
    'Support Requirements.💣 Reactor',
]

generators = read_cards('Generators', fields)
for card in generators:
    if has_movement_modifiers(card):
        intify(card, int_fields)
        boolify(card, bool_fields)
    else:
        card.pop('Movement Modifiers')
        intify(card, filter(lambda f: not f.startswith('Movement Modifiers.'), int_fields))
        boolify(card, filter(lambda f: not f.startswith('Movement Modifiers.'), bool_fields))
    card['Support Requirements'] = supports(card)
    card['Supports'] = [k for k in card['Supports'].keys() if card['Supports'][k]]

draw(generators)

{'Name': 'H2-O2 Fuel Cell',
 'Spectral Type': 'C',
 'Mass': 4,
 'Rad-Hard': 4,
 'Air Eater': False,
 'Ability': '',
 'Supports': ['e'],
 'Support Requirements': [],
 'Deck': 'Generators',
 'Side': 'Front'}

## GW Thrusters

In [11]:
'''
GW Thrusters
'''

fields = [
    'Name',
    'Type',
    'Spectral Type',
    'Promotion Colony',
    'Mass',
    'Rad-Hard',
    'Thrust Triangle.Thrust',
    'Thrust Triangle.Fuel Consumption',
    'Thrust Triangle.Afterburn',
    'Support Requirements.⟛ Generator',
    'Support Requirements.e Generator',
    'Support Requirements.X Reactor',
    'Support Requirements.Any Reactor',
    'Support Requirements.Cooling',
    'Future',
]

int_fields = [
    'Mass',
    'Rad-Hard',
    'Thrust Triangle.Thrust',
    'Support Requirements.Cooling',
    'Thrust Triangle.Afterburn',
]

bool_fields = [
    'Support Requirements.⟛ Generator',
    'Support Requirements.e Generator',
    'Support Requirements.X Reactor',
    'Support Requirements.Any Reactor',
]

gwthrusters = read_cards('GW Thrusters', fields)
for card in gwthrusters:
    intify(card, int_fields)
    boolify(card, bool_fields)
    card['Support Requirements'] = supports(card)
    card['Thrust Triangle']['Fuel Type'] = 'Isotope'

draw(gwthrusters)

{'Name': 'Dense Plasma H-B Focus Fusion',
 'Type': 'GW Thruster',
 'Spectral Type': 'D',
 'Promotion Colony': 'D',
 'Mass': 1,
 'Rad-Hard': 8,
 'Future': '',
 'Thrust Triangle': {'Thrust': 2,
  'Fuel Consumption': '1/10',
  'Afterburn': 5,
  'Fuel Type': 'Isotope'},
 'Support Requirements': [{'Type': 'Generator', 'Subtypes': ['⟛']}],
 'Deck': 'GW Thrusters',
 'Side': 'Front'}

## Freighters

In [12]:
'''
Freighters
'''

fields = [
    'Name',
    'Type',
    'Spectral Type',
    'Promotion Colony',
    'Mass',
    'Rad-Hard',
    'Load-Limit',
    'Factory Loading Only',
    'Bonus Pivots',
    'Supports.⟛ Generator',
    'Supports.e Generator',
    'Supports.X Reactor',
    'Supports.∿ Reactor',
    'Ability',
    'Future',
]

int_fields = [
    'Mass',
    'Rad-Hard',
    'Load-Limit',
    'Bonus Pivots',
]

bool_fields = [
    'Factory Loading Only',
    'Supports.⟛ Generator',
    'Supports.e Generator',
    'Supports.X Reactor',
    'Supports.∿ Reactor',
]

freighters = read_cards('Freighters', fields)
for card in freighters:
    intify(card, int_fields)
    boolify(card, bool_fields)
    card['Support Requirements'] = supports(card)
    card['Supports'] = [k for k in card['Supports'].keys() if card['Supports'][k]]

draw(freighters)

{'Name': 'Inflatable Solar-Heated',
 'Type': 'Freighter',
 'Spectral Type': 'Any',
 'Promotion Colony': 'C',
 'Mass': 0,
 'Rad-Hard': 5,
 'Load-Limit': 2,
 'Factory Loading Only': True,
 'Bonus Pivots': 0,
 'Ability': 'SOLAR HEATED: If not using Powersat, may move out only as far as the Ceres zone.',
 'Future': '',
 'Supports': [],
 'Deck': 'Freighters',
 'Side': 'Front',
 'Support Requirements': []}

## Bernals

In [13]:
'''
Bernals
'''

fields = [
    'Name',
    'Type',
    'Promotion Colony',
    'Mass',
    'Rad-Hard',
    'Thrust Triangle.Thrust',
    'Thrust Triangle.Fuel Consumption',
    'Thrust Triangle.Push',
    'Support Requirements.e Generator',
    'Support Requirements.Cooling',
    'Ability',
]

int_fields = [
    'Mass',
    'Rad-Hard',
    'Thrust Triangle.Thrust',
    'Support Requirements.Cooling',
]

bool_fields = [
    'Thrust Triangle.Push',
    'Support Requirements.e Generator',
]

bernals = read_cards('Bernals', fields)
for card in bernals:
    intify(card, int_fields)
    boolify(card, bool_fields)
    card['Support Requirements'] = supports(card)
    card['Thrust Triangle']['Fuel Type'] = 'Dirt'

draw(bernals)

{'Name': 'Tourism Cycler',
 'Type': 'Bernal',
 'Promotion Colony': 'Atmospheric',
 'Mass': 10,
 'Rad-Hard': 7,
 'Ability': 'HOME: Can designate any Spacecraft to forgo Belt Rolls in the Radiation Belts near Earth.',
 'Thrust Triangle': {'Thrust': 3,
  'Fuel Consumption': '3',
  'Push': True,
  'Fuel Type': 'Dirt'},
 'Support Requirements': [{'Type': 'Generator', 'Subtypes': ['e']}],
 'Deck': 'Bernals',
 'Side': 'Front'}

## Colonists

In [14]:
from functools import reduce
import unittest

def all_f(*fs):
    return lambda *args, **kwargs: all([f(*args, **kwargs) for f in fs])

class FilterTests(unittest.TestCase):
    
    def test_filtering(self):
        fields = [
            'Name',
            'Type',
            'Thrust Triangle.Thrust',
            'Thrust Triangle.Fuel Consumption',
            'Supports.X Reactor',
            'Supports.∿ Reactor',
            'Supports.💣 Reactor',
            'Ability',
        ]
        
        expected_fields = [
            'Name',
            'Type',
            'Ability',
        ]
        
        filter_functions = [lambda x: not x.startswith('Thrust Triangle.'), lambda x: not x.startswith('Supports.')]
        
        filtered_fields = list(filter(all_f(*filter_functions), fields))
        self.assertEqual(filtered_fields, expected_fields)
    
unittest.main(argv=['', 'FilterTests'], verbosity=2, exit=False)

test_filtering (__main__.FilterTests) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


In [15]:
'''
Colonists
'''

from functools import reduce

fields = [
    'Name',
    'Type',
    'Specialty',
    'Spectral Type',
    'Promotion Colony',
    'Ideology',
    'Mass',
    'Rad-Hard',
    'Thrust Triangle.Thrust',
    'Thrust Triangle.Fuel Consumption',
    'Thrust Triangle.Fuel Type',
    'Thrust Triangle.Afterburn',
    'Thrust Triangle.Push',
    'Thrust Triangle.Solar',
    'Air Eater',
    'Bonus Pivots',
    'ISRU.ISRU',
    'ISRU.Missile',
    'ISRU.Raygun',
    'ISRU.Buggy',
    'Supports.X Reactor',
    'Supports.∿ Reactor',
    'Supports.💣 Reactor',
    'Ability',
    'Future',
]

int_fields = [
    'Mass',
    'Rad-Hard',
    'Thrust Triangle.Thrust',
    'Thrust Triangle.Afterburn',
    'ISRU.ISRU',
    'Bonus Pivots',
]

bool_fields = [
    'ISRU.Missile',
    'ISRU.Raygun',
    'ISRU.Buggy',
    'Thrust Triangle.Push',
    'Thrust Triangle.Solar',
    'Air Eater',
    'Supports.X Reactor',
    'Supports.∿ Reactor',
    'Supports.💣 Reactor',
]

def has_isru(card):
    return card.get('ISRU', {}).get('ISRU', '') != ''

def na_to_empty(card):
    na_fields = [f for f, v in card.items() if v == 'n/a']
    for f in na_fields:
        card[f] = ''

def get_multirow_values(card):
    return {
        'Type': card['Type'],
        'Specialty': card['Specialty'],
        'Promotion Colony': card['Promotion Colony'],
        'Ideology': card['Ideology']
    }

def set_multirow_values(card, values):
    for k, v in values.items():
        card[k] = v
    
colonists = read_cards('Colonists', fields)
for i, card in enumerate(colonists):
    if i % 2 == 0:
        multirow_values = get_multirow_values(card)
    if i % 2 == 1:
        set_multirow_values(card, multirow_values)
        
    filters = []
    if not has_thruster(card):
        card.pop('Thrust Triangle')
        filters.append(lambda x: not x.startswith('Thrust Triangle.'))
    this_has_isru = has_isru(card)
    if not this_has_isru:
        card.pop('ISRU')
        filters.append(lambda x: not x.startswith('ISRU.'))
    
    na_to_empty(card)
    intify(card, filter(all_f(*filters), int_fields))
    boolify(card, filter(all_f(*filters), bool_fields))
    card['Supports'] = [k for k in card['Supports'].keys() if card['Supports'][k]]
    
    if this_has_isru:
        card['ISRU'] = isru(card)
    
    if card['Type'] == 'Robot':
        card.pop('Ideology')
    elif card['Type'] == 'Human':
        card.pop('Spectral Type')

draw(colonists)

{'Name': 'Rental Body Guild',
 'Type': 'Human',
 'Specialty': 'Prospector',
 'Promotion Colony': 'H',
 'Ideology': 'Purple',
 'Mass': 1,
 'Rad-Hard': 6,
 'Air Eater': True,
 'Bonus Pivots': 0,
 'Ability': '-1 to Colocated size rolls.',
 'Future': 'ET LIFE FUTURE: Req = Have 2 or more Astrobiological Colonies. Effect (Endgame): +2 VP per Astrobiological Colony.',
 'Thrust Triangle': {'Thrust': 12,
  'Fuel Consumption': '4',
  'Fuel Type': 'Water',
  'Afterburn': 1,
  'Push': False,
  'Solar': False},
 'ISRU': {'Rating': 2, 'Platforms': ['Missile']},
 'Supports': ['∿ Reactor'],
 'Deck': 'Colonists',
 'Side': 'Reverse'}

# Validation

## Aggregating card lists

In [16]:
from itertools import chain

all_cards = list(chain(thrusters, robonauts, refineries, reactors, radiators, generators, gwthrusters, freighters, bernals, colonists))

## "Leaf" schemas and validation

These schemas are "leaves" in the sense that they don't refer to any other schemas.

In [1]:
from jsonschema import validate, ValidationError
import json

thrust_triangle_schema = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'Thrust Triangle',
    'description': 'A thrust triangle on a card.',
    'type': 'object',
    'properties': {
        'Thrust': {
            'description': 'The base thrust of the thrust triangle',
            'type': 'integer',
        },
        'Fuel Consumption': {
            'description': 'A string representation of the base fuel consumption as an integer or fraction.',
            'type': 'string',
        },
        'Fuel Type': {
            'description': 'The type of fuel the thrust triangle consumes.',
            'enum': ['Water', 'Dirt', 'Isotope'],
        },
        'Afterburn': {
            'description': 'The number of afterburns the thruster can perform.',
            'type': 'integer',
        },
        'Push': {
            'description': 'Whether the thrust triangle is pushable.',
            'type': 'boolean',
        },
        'Solar': {
            'description': 'Whether the thrust triangle is solar.',
            'type': 'boolean',
        }
    },
    'required': ['Thrust', 'Fuel Consumption', 'Fuel Type'],
}

isru_schema = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'ISRU',
    'description': "The card's ISRU rating and platform icons.",
    'type': 'object',
    'properties': {
        'Rating': {
            'description': 'The ISRU rating.',
            'type': 'integer',
        },
        'Platforms': {
            'description': 'An array of platforms icons on the card.',
            'type': 'array',
            'items': {
                'enum': ['Raygun', 'Buggy', 'Missile'],                
            },
        },
    },
    'required': ['Rating', 'Platforms'],
}

generator_support_requirement = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'Generator Support Requirement',
    'description': "A generator support required by a card.",
    'properties': {
        'Type': { 'const': 'Generator' },
        'Subtypes': {
            'type': 'array',
            'items': { 'enum': ['e', '⟛'] }
        },
    },
}

reactor_support_requirement = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'Reactor Support Requirement',
    'description': "A reactor support required by a card.",    
    'properties': {
        'Type': { 'const': 'Reactor' },
        'Subtypes': {
            'type': 'array',
            'items': { 'enum': ['X', '∿', '💣'] }
        },
    },
}

cooling_support_requirement = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'Cooling Support Requirement',
    'description': "A cooling support indicating therms of cooling required by a card.",
    'properties': {
        'Type': { 'const': 'Cooling' },
        'Therms': { 'type': 'integer' },
    },
}

with open('schema/thrust-triangle.schema.json', 'w') as f:
    json.dump(thrust_triangle_schema, f)
    
with open('schema/isru.schema.json', 'w') as f:
    json.dump(isru_schema, f)
    
with open('schema/generator-support-requirement.schema.json', 'w') as f:
    json.dump(generator_support_requirement, f)
    
with open('schema/reactor-support-requirement.schema.json', 'w') as f:
    json.dump(reactor_support_requirement, f)
    
with open('schema/cooling-support-requirement.schema.json', 'w') as f:
    json.dump(cooling_support_requirement, f)

## Thruster validation

In [24]:
from jsonschema import validate, ValidationError
import json

thruster_schema = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'Thruster',
    'description': "A thruster in the Thruster deck.",
    'type': 'object',
    '$defs': {
        'thrustTriangle': thrust_triangle_schema,
        'generatorSupportRequirement': generator_support_requirement,
        'reactorSupportRequirement': reactor_support_requirement,
        'coolingSupportRequirement': cooling_support_requirement,
    },
    'properties': {
        'Name': { 'type': 'string' },
        'Deck': { 'const': 'Thrusters' },
        'Side': { 'enum': [ 'Front', 'Reverse' ] },
        'Spectral Type': {
            'enum': ['C', 'S', 'M', 'V', 'D', 'H'],
        },
        'Mass': { 'type': 'integer' },
        'Rad-Hard': { 'type': 'integer' },
        'Thrust Triangle': {
            '$ref': '#/$defs/thrustTriangle',
        },
        'Support Requirements': {
            'type': 'array',
            'items': {
                'oneOf': [
                    { '$ref': '#/$defs/generatorSupportRequirement' },
                    { '$ref': '#/$defs/reactorSupportRequirement' },
                    { '$ref': '#/$defs/coolingSupportRequirement' },
                ]
            },
        },
        'Bonus Pivots' : { 'type': 'integer' },
        'Ability': { 'type': 'string' },
    },
    'required': ['Name', 'Deck', 'Mass', 'Rad-Hard', 'Thrust Triangle', 'Support Requirements'],
    'additionalProperties': False,
}

for card in thrusters:
    try:
        validate(card, thruster_schema)
    except ValidationError as e:
        print('Error validating %s: %s' % (card['Deck'],  card['Name']))
        print(json.dumps(card, ensure_ascii=False, indent=4))
        raise e
print('ok')

draw(thrusters)

ok


{'Name': 'Mass Driver',
 'Spectral Type': 'M',
 'Mass': 3,
 'Rad-Hard': 8,
 'Bonus Pivots': 0,
 'Ability': '',
 'Thrust Triangle': {'Thrust': 4,
  'Fuel Consumption': '3',
  'Fuel Type': 'Dirt',
  'Afterburn': 0,
  'Push': True,
  'Solar': False},
 'Support Requirements': [{'Type': 'Generator', 'Subtypes': ['⟛']}],
 'Deck': 'Thrusters',
 'Side': 'Front'}

In [19]:
bad_card = {'Name': 'Monoatomic Plug Nozzle',
 'Spectral Type': 'M',
 'Mass': 0,
 'Rad-Hard': 6,
 'Bonus Pivots': 0,
 'Ability': '',
 'Thrust Triangle': {'Thrust': 4,
  'Fuel Consumption': '3',
  'Fuel Type': 'Water',
  'Afterburn': 1,
  'Push': False,
  'Solar': False},
 'Support Requirements': [{'Type': 'Reactor', 'Subtypes': ['OHNO']}],
 'Deck': 'Thrusters',
 'Side': 'Front'}

#validate(bad_card, thruster_schema)

In [20]:
from jsonschema import validate

schema_a = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'A',
    'type': 'object',
    'properties': {
        'value': { 'const': 'A' },
    },
}

validate({ 'value': 'A' }, schema_a)

schema_b = {
    'schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'B',
    'type': 'object',
    'properties': {
        'value': { 'const': 'B' },
    },
}

validate({ 'value': 'B' }, schema_b)

either_schema = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'AorB',
    'type': 'object',
    '$defs': {
        'A': schema_a,
        'B': schema_b,
    },
    'oneOf': [
        { '$ref': '#/$defs/A' },
        { '$ref': '#/$defs/B' },
    ]
}

validate({ 'value': 'A' }, either_schema)

## Patent validation

In [51]:
from jsonschema import validate

thruster_patent_schema = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'Thruster Patent',
    'description': 'A card from the Thruster patent deck',
    '$defs': {
    },
    'properties': {
        'Deck': { 'const': 'Thrusters' },
        'Front': { '$ref': '#/$defs/thruster' },
        'Reverse': { '$ref': '#/$defs/thruster' },
    },
    'required': ['Deck', 'Front', 'Reverse'],
    'additionalProperties': False,
}

thruster_patent_deck_schema = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'Thruster Patent Deck',
    'description': "The thruster patent deck.",
    '$defs': {
        'thrusterPatent': thruster_patent_schema,
        'thruster': thruster_schema,
        'thrustTriangle': thrust_triangle_schema,
        'generatorSupportRequirement': generator_support_requirement,
        'reactorSupportRequirement': reactor_support_requirement,
        'coolingSupportRequirement': cooling_support_requirement,
    },
    'properties': {
        'Name': { 'type': 'string' },
        'Patents': {
            'type': 'array',
            'items': { '$ref': '#/$defs/thrusterPatent' }
        }
    },
    'required': ['Name', 'Patents'],
    'additionalProperties': False,
}

patent_deck_schema = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'Patent Deck',
    'description': "A patent deck.",
    '$defs': {
        'thrusterPatentDeck': thruster_patent_deck_schema,
        'thrusterPatent': thruster_patent_schema,
        'thruster': thruster_schema,
        'thrustTriangle': thrust_triangle_schema,
        'generatorSupportRequirement': generator_support_requirement,
        'reactorSupportRequirement': reactor_support_requirement,
        'coolingSupportRequirement': cooling_support_requirement,
    },
    'oneOf': [
        { '$ref': '#/$defs/thrusterPatentDeck' },
    ],
}

validate(thruster_deck, patent_deck_schema)

Hmm this definitely seems like the point at which URIs become somewhat necessary...